<a href="https://colab.research.google.com/github/DataScienceUB/DeepLearningMaster20192020/blob/master/3-Tensorflow%20programming%20model%20(Solution).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Exercise

Let's built and visualize and complex model:

+ Our inputs will be placeholders.
+ The model will take in a single vector of any lenght.
+ The graph will be segmented in name scopes.
+ We will accumulate the total value of all outputs over time.
+ At each run, we are going to save the output of the graph, the accumulated total of all outputs, and the average value of all outputs to disk for use in `tensorboard`.


![alt text](https://github.com/DataScienceUB/DeepLearningMaster20192020/blob/master/images/t6.png?raw=1)


In [0]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

# Explicitly create a Graph object
graph = tf.Graph()

with graph.as_default():
    
    with tf.name_scope("variables"):
        # Variable to keep track of how many times the graph has been run
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")

        # Variable that keeps track of the sum of all output values over time:
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="total_output")

    with tf.name_scope("transformation"):

        # Separate input layer
        with tf.name_scope("input"):
            # Create input placeholder- takes in a Vector
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")

        # Separate middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_sum(a, name="sum_b")
            c = tf.reduce_prod(a, name="product_c")

        # Separate output layer
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
        
        
    with tf.name_scope("update"):
        # Increments the total_output Variable by the latest output
        update_total = total_output.assign_add(output)

        # Increments the above `global_step` Variable, should be run whenever the graph is run
        increment_step = global_step.assign_add(1)

    
    # Summary Operations
    with tf.name_scope("summaries"):
        # Calculating average (avg = total/steps)
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")

        # Creates summaries for output node
        tf.summary.scalar("output_summary", output)
        tf.summary.scalar("total_summary", update_total)
        tf.summary.scalar("average_summary", avg)
    
    # Global Variables and Operations
    with tf.name_scope("global_ops"):
        # Initialization Op
        init = tf.global_variables_initializer()
        # Merge all summaries[…]
        merged_summaries = tf.summary.merge_all()
       # Start a Session, using the explicitly created Graph
sess = tf.Session(graph=graph)

# Open a SummaryWriter to save summaries
writer = tf.summary.FileWriter('./improved_graph', graph)

# Initialize Variables
sess.run(init)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Let's write a function to run the graph several times:

In [0]:
def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor}
    out, step, summary = sess.run([output, increment_step, merged_summaries], 
                                  feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [0]:
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

# Write the summaries to disk
writer.flush()

# Close the SummaryWriter
writer.close()

# Close the session
sess.close()

To start TensorBoard after running this code, run the following command:

`tensorboard --logdir='./improved_graph'`   

Bibliography: https://www.easy-tensorflow.com/tf-tutorials/basics/introduction-to-tensorboard